In [ ]:
import fastf1
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import math
from shapely.geometry import LineString
from shapely.affinity import translate
import matplotlib.pyplot as plt
import contextily as cx

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
session_load = fastf1.get_session(2023, 'Monza', 'R')
session_load.load(telemetry=True, laps=True, weather=False)

monza_track = gpd.read_file("bacinger f1-circuits master circuits/it-1922.geojson") #This is monza
original_centroid = monza_track.geometry.centroid.iloc[0]

In [ ]:
raw_df = pd.read_csv('flask/static/raw_df.csv')
interpolated_df = pd.read_csv('flask/static/interpolated_df.csv')

In [ ]:
def get_corners(session):
    circuit_info = session.get_circuit_info()
    corners_df = circuit_info.corners
    return corners_df

In [ ]:
def filter_df_to_corner(data,corner,d_threshold):
    data['Distance_to_corner'] = np.sqrt((data['X'] - get_corners(session_load)['X'].iloc[corner]) ** 2 + (data['Y'] - get_corners(session_load)['Y'].iloc[corner]) ** 2)

    threshold_distance = d_threshold

    filtered_df = data[data['Distance_to_corner'] < threshold_distance]

    return filtered_df

In [ ]:
def coordinate_shift(original_centroid, f1_api_coords):
    """This translates the original relative coordinates into longitude and latitude
    original_centroid is the centroid computed from the downloaded track data
    """
    centroid_lon, centroid_lat = (original_centroid.x, original_centroid.y)  


      
    # conversion factors - these are approximations, adjust as necessary  
    # 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
    km_per_degree_lat = 1 / 111  
    km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
    
    # your array of tuples  
    xy_coordinates = f1_api_coords
    
    # convert each tuple in the array  
    lonlat_coordinates = []  
    for y,x in xy_coordinates:  
        lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
        lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
        lonlat_coordinates.append((lon,lat))  
    


    relative_line = LineString(lonlat_coordinates)
    return relative_line



def shift_centroid(relative_line,original_centroid):
    """This shift the centroid computed"""
    # Calculate the distance to translate in each direction  
    # dx = original_centroid.x - relative_line.centroid.x  
    # dy = original_centroid.y - relative_line.centroid.y  
    dx = -0.004091249607403924
    dy = -0.006398742570105753
    # Shift the LineString  
    shifted_line = translate(relative_line, xoff=dx, yoff=dy)  
    return shifted_line

In [ ]:
def folium_plot(centroid,data,plot_type):
    kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='Esri.WorldImagery', attr="Esri",max_zoom=19,maxNativeZoom = 19)
    drivers = set(data['driver'])
    laps = set(data['LapNumber'])

    for lap in laps:
        for driver in drivers:
            if driver in ['22','31']:
                pass
            else:
                data_ = data[(data['driver'] == driver) & (data['LapNumber'] == lap)]
                coords = [(row['Y'],row['X']) for index,row in data_.iterrows()]
                scaled_down = coordinate_shift(centroid,coords)
                shifted_line = shift_centroid(scaled_down,centroid)


                gdf_ = gpd.GeoDataFrame(geometry=[shifted_line], crs="EPSG:4326")    
                new_projected = gdf_.to_crs(epsg=32632)

                style = {'color': 'black', 'weight': 0.4}  # Adjust weight as needed

                folium.GeoJson(new_projected,style=style).add_to(kat)

In [ ]:
test = filter_df_to_corner(interpolated_df,0,1000)

In [ ]:
test = test[~test['driver'].isin(['22','31'])]

In [ ]:
coords = [(row['Y'],row['X']) for index,row in test.iterrows()]
scaled_down = coordinate_shift(original_centroid,coords)
shifted_line = shift_centroid(scaled_down,original_centroid)

In [ ]:
test['shifted_x'] = [x for x,y in shifted_line.coords]
test['shifted_y'] = [y for x,y in shifted_line.coords]

In [ ]:
gdf = gpd.GeoDataFrame(test, geometry=gpd.points_from_xy(test['shifted_x'], test['shifted_y']),crs="EPSG:4326").reset_index(drop=True)

In [ ]:
df_wm = gdf.to_crs(epsg=3857)

In [ ]:
ax = df_wm.plot(column='Speed', legend=True, figsize=(14, 18), cmap='OrRd',markersize = 0.5,alpha = 0.1)
cx.add_basemap(ax,source=cx.providers.Esri.WorldImagery)